The following code has been adapted from https://github.com/computervisioneng/emotion-recognition-python-scikit-learn-mediapipe/tree/main.

In [18]:
import os
import cv2
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC
import mediapipe as mp
import pickle
import pandas as pd
from sklearn.naive_bayes import GaussianNB

In [9]:
# Grabs important facial features from the image

def get_facial_features(image, draw=False, static_image_mode=True):

    # Read the input image
    image_input_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    # Extracts 3D face landmarks from the image by using machine learning to infer the 3D facial surface
    face_mesh = mp.solutions.face_mesh.FaceMesh(static_image_mode=static_image_mode,
                                                max_num_faces=1,
                                                min_detection_confidence=0.5)
    image_rows, image_cols, _ = image.shape
    results = face_mesh.process(image_input_rgb)

    image_landmarks = []

    if results.multi_face_landmarks:

        if draw:

            mp_drawing = mp.solutions.drawing_utils
            mp_drawing_styles = mp.solutions.drawing_styles
            drawing_spec = mp_drawing.DrawingSpec(thickness=2, circle_radius=1)

            mp_drawing.draw_landmarks(
                image=image,
                landmark_list=results.multi_face_landmarks[0],
                connections=mp.solutions.face_mesh.FACEMESH_CONTOURS,
                landmark_drawing_spec=drawing_spec,
                connection_drawing_spec=drawing_spec)

        ls_single_face = results.multi_face_landmarks[0].landmark
        xs_ = []
        ys_ = []
        zs_ = []
        for idx in ls_single_face:
            xs_.append(idx.x)
            ys_.append(idx.y)
            zs_.append(idx.z)
        for j in range(len(xs_)):
            image_landmarks.append(xs_[j] - min(xs_))
            image_landmarks.append(ys_[j] - min(ys_))
            image_landmarks.append(zs_[j] - min(zs_))
    return image_landmarks

In [10]:
# Loads the data

data_directory = "./Emotion-Faces"

output = []
for emotion_index, emotion in enumerate(sorted(os.listdir(data_directory))):
    for directory_image_path in os.listdir(os.path.join(data_directory, emotion)):
        image_path = os.path.join(data_directory, emotion, directory_image_path)
        image = cv2.imread(image_path)
        facial_features = get_facial_features(image)
        if len(facial_features) == 1404:
            facial_features.append(int(emotion_index))
            output.append(facial_features)

np.savetxt('data.txt', np.asarray(output))
print("Data has been loaded and saved in data.txt.")

Data has been loaded and saved in data.txt.


In [11]:
# Train the model on the data

# Load data
data_file = "data.txt"
data = np.loadtxt(data_file)

# Split data into features (X) and labels (Y)
x = data[:, :-1]
y = data[:, -1]

# Split X and Y into training and testing sets.
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42, shuffle=True, stratify=y)

The Multilayer Classifier (MLP) has the best results on identifying emotion, as shown below. 

In [12]:
# Initialize and train the Multilayer Perceptron Classifier

# Stats

# All Emotions: Anger, Disgust, Fear, Happy, Neural Sad, Surprise

# Success rate on all emotions: 67.14%

# Success rate on Anger, Happy, Neutral, and Sad: 85%

mlp_classifier = MLPClassifier(max_iter=1000, random_state=4)

# Train the classifier on the training data
mlp_classifier.fit(x_train, y_train)

# Saves the model
with open('./facial-emotion-model', 'wb') as f:
    pickle.dump(mlp_classifier, f)

# To open the model:
# with open('./facial-emotion-model', 'rb') as f:
#     model = pickle.load(f)

# Evaluate the accuracy of the model
y_predict = mlp_classifier.predict(x_test)
accuracy = accuracy_score(y_test, y_predict)
print(f"Accuracy: {accuracy * 100:.2f}%")

# Print confusion matrix
labels = ['Anger', 'Disgust', 'Fear', 'Happiness', 'Neutral', 'Sadness', 'Surprise']
cm = confusion_matrix(y_test, y_predict)
cm_df = pd.DataFrame(cm, index=labels, columns=labels)
print("Confusion Matrix:")
print(cm_df)

Accuracy: 67.14%
Confusion Matrix:
           Anger  Disgust  Fear  Happiness  Neutral  Sadness  Surprise
Anger          7        0     1          1        1        0         0
Disgust        2        5     0          2        0        1         0
Fear           0        0     6          2        0        0         2
Happiness      0        0     0         10        0        0         0
Neutral        0        1     0          0        8        1         0
Sadness        0        1     0          0        7        2         0
Surprise       1        0     0          0        0        0         9


In [13]:
# Find best random state for MLP

max_accuracy = 0
best_state = 1

for i in range(100):
    mlp_classifier = MLPClassifier(max_iter=1000, random_state=4) # 4 is 85
    
    # Train the classifier on the training data
    mlp_classifier.fit(x_train, y_train)
    
    # Evaluate the accuracy of the model
    y_predict = mlp_classifier.predict(x_test)
    accuracy = accuracy_score(y_test, y_predict)

    if accuracy > max_accuracy:
        max_accuracy = accuracy
        best_state = i

print("Best Accuracy:", max_accuracy)
print("Best State:", best_state)

Best Accuracy: 0.6714285714285714
Best State: 0


The following classifiers were used for testing purposes to see if it exceeded the performance of MLP, which they did not.

In [22]:
# Initialize the Random Forest Classifier
rf_classifier = RandomForestClassifier()

# Train the classifier on the training data
rf_classifier.fit(x_train, y_train)

# Evaluate the accuracy of the model
y_predict = rf_classifier.predict(x_test)
accuracy = accuracy_score(y_test, y_predict)
print(f"Accuracy: {accuracy * 100:.2f}%")

Accuracy: 58.57%


In [23]:
# Initialize the Decision Tree Classifier
dt_classifier = DecisionTreeClassifier()

# Train the classifier on the training data
dt_classifier.fit(x_train, y_train)

# Evaluate the accuracy of the model
y_predict = dt_classifier.predict(x_test)
accuracy = accuracy_score(y_test, y_predict)
print(f"Accuracy: {accuracy * 100:.2f}%")

Accuracy: 51.43%


In [24]:
# Initialize the Support Vector Classifier (SVC)
svm_classifier = SVC()

# Train the classifier on the training data
svm_classifier.fit(x_train, y_train)

# Evaluate the accuracy of the model
y_predict = svm_classifier.predict(x_test)
accuracy = accuracy_score(y_test, y_predict)
print(f"Accuracy: {accuracy * 100:.2f}%")

Accuracy: 42.86%


In [25]:
# Initialize the Gaussian Naive Bayes
gnb_classifier = GaussianNB()

# Train the classifier on the training data
gnb_classifier.fit(x_train, y_train)

# Evaluate the accuracy of the model
y_predict = gnb_classifier.predict(x_test)
accuracy = accuracy_score(y_test, y_predict)
print(f"Accuracy: {accuracy * 100:.2f}%")

Accuracy: 42.86%
